In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
clicks = np.load("../preprocessing/clicks/clicks.npy")

In [3]:
clicks = -1 + 2*clicks 

In [4]:
print(np.sum(np.isnan(clicks)))

1091


In [5]:
print(clicks[17])

[ 1. -1. -1.  1. -1. -1.  1. -1. -1.  1. -1. -1.  1. -1. -1.  1. -1. -1.
  1. -1. -1.  1. -1. -1.  1. -1. -1.  1. -1. -1.  1. -1. -1.  1. -1. -1.
  1. -1. -1.  1. -1. -1.  1. -1. -1.  1. -1. -1.  1. -1. -1.  1. -1. -1.
 -1.  1. -1. -1.  1. -1. -1.  1. -1. -1.  1. -1. -1.  1. -1. -1.  1. -1.
 -1.  1. -1. -1.  1. -1. -1.  1. -1.  1. -1. -1.  1. -1. -1.  1. -1. -1.
  1. -1. -1. -1.  1. -1. -1.  1. -1.  1. -1. -1.  1. -1. -1.  1. -1. -1.
  1. -1. -1.  1. -1. -1.  1. -1. -1.  1. -1. -1.  1. -1. -1.  1. -1.  1.
 -1. -1.  1. -1. -1.  1. -1. -1.  1. -1. -1.  1. -1. -1.  1. -1. -1.  1.
 -1. -1.  1. -1. -1.  1. -1.  1. -1. -1. -1.  1. -1. -1.  1. -1. -1.  1.
 -1. -1.  1. -1. -1.  1. -1. -1.  1. -1. -1.  1. -1. -1.  1. -1. -1.  1.
 -1. -1.  1. -1. -1.  1. -1. -1.  1. -1. -1.  1. -1. -1.  1. -1. -1.  1.
 -1. -1.  1. -1. -1.  1. -1. -1. -1.  1. -1. -1.  1. -1.  1. -1. -1.  1.
 -1. -1.  1. -1. -1.  1. -1. -1.  1. -1. -1.  1. -1. -1.  1. -1. -1.  1.
 -1. -1.  1. -1. -1.  1. -1. -1.  1. -1. -1.  1. -1

In [6]:
info_songs = pd.read_csv("../Info/info.csv")

In [7]:
label_songs = pd.read_csv("../preprocessing/labels.csv")

In [8]:
select_label = "genre"

if select_label == "genre":
    labels = label_songs[select_label].map({"classical":0, "electronic":1, "pop":2, "rock":3})
labels = labels.to_numpy()

In [9]:
length_songs = pd.DataFrame(info_songs["Track ID"])
length_songs["Song length"] = info_songs["Duration"]*10
length_songs["Song length"] = length_songs["Song length"].astype(int)
length_songs

,Track ID,Song length
0,1,600
1,10,600
2,100,600
3,11,600
4,12,600
...,...,...
395,395,600
396,396,600
397,397,600
398,398,600


In [10]:
n_data = len(length_songs)
idx = np.random.permutation(n_data)
length_songs = length_songs.reindex(idx)["Song length"].to_numpy()
clicks = clicks[idx, :]
labels = labels[idx]

In [11]:
import tensorflow as tf

In [12]:
fraction_validation = 0.25
n_test = int(fraction_validation*len(labels))
n_train = len(labels) - n_test

train_features, train_labels, train_length = clicks[:n_train], labels[:n_train], length_songs[:n_train]
validation_features, validation_labels, validation_length = clicks[n_train:], labels[n_train:], length_songs[n_train:]

In [13]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_features, train_labels, train_length))
validation_dataset = tf.data.Dataset.from_tensor_slices((validation_features, validation_labels, validation_length))

In [21]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras import Input, layers
from tensorflow.keras import backend as K

sample_size = 100

drop_out_rate = 0.2

# very simple keras Sequential model
input_tensor = Input((sample_size,1))
x = layers.Conv1D(16, 50, padding="valid", activation="relu", strides=1)(input_tensor)
x = layers.Conv1D(32, 5, padding="valid", activation="relu", strides=1)(x)
x = layers.Dropout(drop_out_rate)(x)
x = layers.MaxPooling1D(strides=2)(x)
x = layers.Conv1D(64, 5, padding="valid", activation="relu", strides=1)(x)
x = layers.MaxPooling1D(strides=2)(x)
x = layers.Flatten()(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Flatten()(x)
x = layers.Dropout(drop_out_rate)(x)
output_tensor = layers.Dense(4, activation="softmax")(x)

model = tf.keras.Model(input_tensor, output_tensor)

In [22]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 100, 1)]          0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 51, 16)            816       
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 47, 32)            2592      
_________________________________________________________________
dropout_3 (Dropout)          (None, 47, 32)            0         
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 23, 32)            0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 19, 64)            10304     
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 9, 64)             0   

In [23]:
model.compile(optimizer="adam",
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [24]:
def slice_accordingly(input_tensor, labels, size=sample_size):

    input_shape = (tf.shape(input_tensor).numpy())
    
    input_shape[1] = size
    size = input_shape
    
    sliced_tensor = tf.image.random_crop(input_tensor, size)
    
    nan_values, idx = tf.unique(tf.gather(tf.where(tf.math.is_nan(sliced_tensor)), 0, axis=1))
    nan_values = nan_values.numpy()
  
    msk = np.zeros((input_shape[0]), dtype=np.bool)
    msk[nan_values] = True
    msk = ~msk

    sliced_tensor = tf.boolean_mask(sliced_tensor, msk , axis=0)
    labels = tf.boolean_mask(labels , msk, axis=0)
    
    return sliced_tensor, labels

In [25]:
n_epochs = 100
batch_size = 50

for epoch in range(n_epochs):
    train_ds = train_dataset.shuffle(n_train).batch(batch_size)
    val_ds = validation_dataset.shuffle(n_test).batch(n_test)
    
    x_val, y_val, length_val = next(iter(val_ds))
    x_val, y_val = slice_accordingly(x_val, y_val)
    
    for features, labels, lengths in train_ds:
        features, labels = slice_accordingly(features, labels)
        
        history = model.fit(features, labels,
                           validation_data=(x_val, y_val)) 
        predictions = model.predict(features)

2/2 [==============================] - 0s 81ms/step - loss: 0.9865 - accuracy: 0.6600 - val_loss: 1.4124 - val_accuracy: 0.3300
